In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


In [2]:
df = pd.read_csv('data/combined_subjects7.csv', index_col=0)
subject_id_list = df['subject'].unique()


In [3]:
feats_all =   ['BVP_mean', 'BVP_std', 'BVP_min', 'BVP_max',
           'EDA_phasic_mean', 'EDA_phasic_std', 'EDA_phasic_min', 'EDA_phasic_max', 'EDA_smna_mean',
           'EDA_smna_std', 'EDA_smna_min', 'EDA_smna_max', 'EDA_tonic_mean',
           'EDA_tonic_std', 'EDA_tonic_min', 'EDA_tonic_max', 'Resp_mean',
           'Resp_std', 'Resp_min', 'Resp_max', 'TEMP_mean', 'TEMP_std', 'TEMP_min',
           'TEMP_max', 'TEMP_slope', 'BVP_peak_freq','BVP_peaks_cnt','Resp_peak_cnt','subject', 'label']

feats_without_EDA =   ['BVP_mean', 'BVP_std', 'BVP_min', 'BVP_max','Resp_mean','Resp_std', 'Resp_min', 'Resp_max', 'TEMP_mean', 'TEMP_std', 'TEMP_min',
           'TEMP_max', 'TEMP_slope', 'BVP_peak_freq','BVP_peaks_cnt','Resp_peak_cnt','subject', 'label']

feats_EDA = ['EDA_phasic_mean', 'EDA_phasic_std', 'EDA_phasic_min', 'EDA_phasic_max', 'EDA_smna_mean',
           'EDA_smna_std', 'EDA_smna_min', 'EDA_smna_max', 'EDA_tonic_mean',
           'EDA_tonic_std', 'EDA_tonic_min', 'EDA_tonic_max','subject', 'label']
df=df[feats_all]

In [4]:
subjects = df['subject'].unique()

In [5]:
x=df.drop(columns=['label'])
y=df['label']

In [6]:
def get_binary_label(label):
    if label == 0 or label == 1:
        return 0
    else:
        return 1

In [7]:
y = y.apply(get_binary_label)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 25)

In [9]:
#params={'C':np.linspace(51,70,20)}
params={'C':[0.001,0.1,1,5,10,15,20,30,40,50,60,70,80,90,100,110,150]}
model = LogisticRegression()
clf = GridSearchCV(model,param_grid=params,cv=3)

In [10]:
lr=clf.fit(X_train,y_train)

In [11]:
best_C=lr.best_params_['C']
best_C

50

In [12]:
preds={}
clr={}
for i in subjects:
    x_train=x[x['subject']!=i]
    x_test=x[x['subject']==i]
    y_train=y[x['subject']!=i]
    y_test=y[x['subject']==i]
    
    clf = LogisticRegression(C=best_C)
    
    best=clf.fit(x_train, y_train)
    
    y_predicted = best.predict(x_test)
    
    print("Subject ",i)
     
    preds[i]=y_predicted
    print(classification_report(y_predicted,y_test))
    a=classification_report(y_predicted,y_test,target_names=['Non-Stress','Stress'],output_dict=True)
    clr[i]=a
    

Subject  2
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       165
           1       0.83      0.86      0.85        64

    accuracy                           0.91       229
   macro avg       0.89      0.90      0.89       229
weighted avg       0.91      0.91      0.91       229

Subject  3
              precision    recall  f1-score   support

           0       0.66      0.91      0.76       117
           1       0.86      0.51      0.64       115

    accuracy                           0.72       232
   macro avg       0.76      0.71      0.70       232
weighted avg       0.75      0.72      0.70       232

Subject  4
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       165
           1       1.00      1.00      1.00        68

    accuracy                           1.00       233
   macro avg       1.00      1.00      1.00       233
weighted avg       1.00      1.00      1.

In [13]:
acc=0
nonstress_f1=0
stress_f1=0
for i in subjects:
    acc=acc+clr[i]['accuracy']
    nonstress_f1=nonstress_f1+clr[i]['Non-Stress']['f1-score']
    stress_f1=stress_f1+clr[i]['Stress']['f1-score']

print("Average Accuracy : ",acc/len(subjects))
print("F1 score for Non Stress : ",nonstress_f1/len(subjects))
print("F1 score for Stress : ",stress_f1/len(subjects))

Average Accuracy :  0.9104972006650006
F1 score for Non Stress :  0.9346594215564945
F1 score for Stress :  0.8509214561271722
